In [1]:
import findspark
findspark.init('/home/murali/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Joins').getOrCreate()
import pandas as pd

23/04/04 11:59:11 WARN Utils: Your hostname, murali-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/04 11:59:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 11:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
emp_path = '/home/murali/Desktop/data_files/emp.csv'

emp = spark.read.csv(emp_path,header=True, inferSchema=True)
emp.show()
emp.printSchema()

dept_path = '/home/murali/Desktop/data_files/dept.csv'
dept = spark.read.csv(dept_path,header=True, inferSchema=True)
dept.show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

In [3]:
from pyspark.sql.functions import count,min,max,sum
empByDeptJob = emp.groupBy('DEPTNO','JOB').agg(                                              \
                               count('EMPNO').alias('Number of Persons')                     \
                              )

empByDeptJob.show()

+------+---------+-----------------+
|DEPTNO|      JOB|Number of Persons|
+------+---------+-----------------+
|    20|  ANALYST|                2|
|    20|  MANAGER|                1|
|    30|  MANAGER|                1|
|    30| SALESMAN|                4|
|    30|    CLERK|                1|
|    20|    CLERK|                2|
|    10|PRESIDENT|                1|
|    10|    CLERK|                1|
|    10|  MANAGER|                1|
+------+---------+-----------------+



In [4]:
emp_pivote = emp.groupBy('JOB').pivot('DEPTNO').count()

In [5]:
emp_pivote.show()
emp_pivote = emp_pivote.fillna(0)
emp_pivote.show()

+---------+----+----+----+
|      JOB|  10|  20|  30|
+---------+----+----+----+
|  ANALYST|null|   2|null|
| SALESMAN|null|null|   4|
|    CLERK|   1|   2|   1|
|  MANAGER|   1|   1|   1|
|PRESIDENT|   1|null|null|
+---------+----+----+----+

+---------+---+---+---+
|      JOB| 10| 20| 30|
+---------+---+---+---+
|  ANALYST|  0|  2|  0|
| SALESMAN|  0|  0|  4|
|    CLERK|  1|  2|  1|
|  MANAGER|  1|  1|  1|
|PRESIDENT|  1|  0|  0|
+---------+---+---+---+



# Unpivote :


we don't have unpivote directly. the option is to use stack()


In [6]:
from pyspark.sql.functions import expr

In [7]:
data = [
    ('ANALYST',None, 2,0),
    ('SALESMAN',None,None,4),
    ('CLERK',1,2,1),
    ('MANAGER',1,1,1),
    ('PRESIDENT',1,None,None)    
]
schema = ('job','d10','d20','d30')

emp_pivote = spark.createDataFrame(data,schema=schema) 
emp_pivote.show()
emp_pivote = emp_pivote.fillna(0)
emp_pivote = emp_pivote.select('JOB',expr("stack(3,'ACCOUNTING',d10,'RESEARCH',d20,'SALES',d30) as (gender,count)"))

emp_pivote.show()

+---------+----+----+----+
|      job| d10| d20| d30|
+---------+----+----+----+
|  ANALYST|null|   2|   0|
| SALESMAN|null|null|   4|
|    CLERK|   1|   2|   1|
|  MANAGER|   1|   1|   1|
|PRESIDENT|   1|null|null|
+---------+----+----+----+

+---------+----------+-----+
|      JOB|    gender|count|
+---------+----------+-----+
|  ANALYST|ACCOUNTING|    0|
|  ANALYST|  RESEARCH|    2|
|  ANALYST|     SALES|    0|
| SALESMAN|ACCOUNTING|    0|
| SALESMAN|  RESEARCH|    0|
| SALESMAN|     SALES|    4|
|    CLERK|ACCOUNTING|    1|
|    CLERK|  RESEARCH|    2|
|    CLERK|     SALES|    1|
|  MANAGER|ACCOUNTING|    1|
|  MANAGER|  RESEARCH|    1|
|  MANAGER|     SALES|    1|
|PRESIDENT|ACCOUNTING|    1|
|PRESIDENT|  RESEARCH|    0|
|PRESIDENT|     SALES|    0|
+---------+----------+-----+



In [8]:
data = [
        ('IT',8,5),
        ('Payroll',None,2),
        ('HR',2,None)    
]

schema = ['dep','male','female']

df = spark.createDataFrame(data,schema)
df.show()
unpivotDF = df.select('dep',expr("stack(2,'male',male,'female',female) as (gender,count)"))
unpivotDF.show()

+-------+----+------+
|    dep|male|female|
+-------+----+------+
|     IT|   8|     5|
|Payroll|null|     2|
|     HR|   2|  null|
+-------+----+------+

+-------+------+-----+
|    dep|gender|count|
+-------+------+-----+
|     IT|  male|    8|
|     IT|female|    5|
|Payroll|  male| null|
|Payroll|female|    2|
|     HR|  male|    2|
|     HR|female| null|
+-------+------+-----+



In [9]:
data = [
    
    (1,'murali','male',1000,None),
    (2,'geetha','female',None,'it'),
    (3,'abcd',None,1000,'Hr')
]

schema = ('id','name','gender','dep')

df = spark.createDataFrame(data,schema)

df.show()


+---+------+------+----+----+
| id|  name|gender| dep|  _5|
+---+------+------+----+----+
|  1|murali|  male|1000|null|
|  2|geetha|female|null|  it|
|  3|  abcd|  null|1000|  Hr|
+---+------+------+----+----+



In [10]:
df.fillna('Unknown').show() # this fill all text fields with specified  value 'Unknown' in data frame
df.fillna(0).show() # since it is numaric values we are asking to fill where there are nulls, it is attempting to fill in numaric fields
df.na.fill('Unknown',['gender','dep']).show() # this replaces the null value with 'Unknown' in gender and dep fields


+---+------+-------+----+-------+
| id|  name| gender| dep|     _5|
+---+------+-------+----+-------+
|  1|murali|   male|1000|Unknown|
|  2|geetha| female|null|     it|
|  3|  abcd|Unknown|1000|     Hr|
+---+------+-------+----+-------+

+---+------+------+----+----+
| id|  name|gender| dep|  _5|
+---+------+------+----+----+
|  1|murali|  male|1000|null|
|  2|geetha|female|   0|  it|
|  3|  abcd|  null|1000|  Hr|
+---+------+------+----+----+

+---+------+-------+----+----+
| id|  name| gender| dep|  _5|
+---+------+-------+----+----+
|  1|murali|   male|1000|null|
|  2|geetha| female|null|  it|
|  3|  abcd|Unknown|1000|  Hr|
+---+------+-------+----+----+



# sample()

to get the random sampling subset from the large dataset

Use function to indicate what percentage of data to return and seed value to make sure every time to get same random sample

In [11]:
# This allows us to get sample from a data frame for profiling. 
#if you provide seed which gives same data set as a sample always
df = spark.range(start = 1, end = 101)
df1 = df.sample(fraction=0.1,seed = 123)
df2 = df.sample(fraction=0.1,seed=123)

df1.show()
df2.show()

+---+
| id|
+---+
| 36|
| 39|
| 42|
| 46|
| 72|
| 85|
| 88|
|100|
+---+

+---+
| id|
+---+
| 36|
| 39|
| 42|
| 46|
| 72|
| 85|
| 88|
|100|
+---+




# Transform()

it's is used ot chain the custom transformations and this function returns the new DataFrame after applying the specified transformations.

In [12]:
from pyspark.sql.functions import upper
data = [
    
    (1,'murali','male',1000,'IT'),
    (2,'geetha','female',2000,'it'),
    (3,'Chintu','male',1000,'Hr')
]

schema = ('id','name','gender','salary','dep')

df = spark.createDataFrame(data,schema)

df.show()

def convertToUpper(df):
    return df.withColumn('name',upper(df.name))

def doubleTheSal(df):
    return df.withColumn('salary',df.salary * 2)

df1 = df.transform(convertToUpper)
df1.show()

df1 = df.transform(doubleTheSal)
df1.show()

+---+------+------+------+---+
| id|  name|gender|salary|dep|
+---+------+------+------+---+
|  1|murali|  male|  1000| IT|
|  2|geetha|female|  2000| it|
|  3|Chintu|  male|  1000| Hr|
+---+------+------+------+---+

+---+------+------+------+---+
| id|  name|gender|salary|dep|
+---+------+------+------+---+
|  1|MURALI|  male|  1000| IT|
|  2|GEETHA|female|  2000| it|
|  3|CHINTU|  male|  1000| Hr|
+---+------+------+------+---+

+---+------+------+------+---+
| id|  name|gender|salary|dep|
+---+------+------+------+---+
|  1|murali|  male|  2000| IT|
|  2|geetha|female|  4000| it|
|  3|Chintu|  male|  2000| Hr|
+---+------+------+------+---+



In [13]:
df1.show()


+---+------+------+------+---+
| id|  name|gender|salary|dep|
+---+------+------+------+---+
|  1|murali|  male|  2000| IT|
|  2|geetha|female|  4000| it|
|  3|Chintu|  male|  2000| Hr|
+---+------+------+------+---+



In [26]:
pdf

,id,name,gender,salary,dep
0,1,murali,male,2000,IT
1,2,geetha,female,4000,it
2,3,Chintu,male,2000,Hr


# createOrReplaceTempView()

A temporary view is a way to create a pointer or reference to a DataFrame or Dataset that can be used to query the data using SQL-like syntax. The view is temporary and exists only for the duration of the Spark session, and can be accessed from any SparkSession connected to the same cluster.

In [4]:
emp_path = '/home/murali/Desktop/data_files/emp.csv'

emp = spark.read.csv(emp_path,header=True, inferSchema=True)
emp.show()
#emp.printSchema()

dept_path = '/home/murali/Desktop/data_files/dept.csv'
dept = spark.read.csv(dept_path,header=True, inferSchema=True)
dept.show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

In [5]:
emp.createOrReplaceTempView('employees')
dept.createOrReplaceTempView('department')

In [6]:
df1 = spark.sql('select employees.JOB, \
                        sum(employees.SAL) as Total_SAL \
                FROM employees WHERE employees.DEPTNO IN (10,20) \
                GROUP BY employees.JOB HAVING sum(employees.sal) >= 5000 \
                ')

df1.show()

+---------+---------+
|      JOB|Total_SAL|
+---------+---------+
|  ANALYST|     6000|
|  MANAGER|     5425|
|PRESIDENT|     5000|
+---------+---------+



In [7]:
dept.createOrReplaceTempView('department')

spark.sql('select e.JOB,d.DNAME , sum(e.sal) as salary \
FROM employees e left join \
department d \
on e.deptno = d.deptno \
WHERE e.DEPTNO IN (10,20)\
group by e.JOB,d.DNAME \
having sum(e.sal) >= 5000').show()

+---------+----------+------+
|      JOB|     DNAME|salary|
+---------+----------+------+
|  ANALYST|  RESEARCH|  6000|
|PRESIDENT|ACCOUNTING|  5000|
+---------+----------+------+



In [16]:
emp.createOrReplaceGlobalTempView('empGlobal')
dept.createOrReplaceGlobalTempView('deptGlobal')

# when you create a global temp view, we need to use global_tem.tableName to access the table
# this global temp view allows us to access this table across the spark sessions
# All global Temp views will be created under global_temp name space.

In [13]:
spark.sql('select * from global_temp.empGlobal').show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

In [17]:
# list tables from current session
spark.catalog.listTables(spark.catalog.currentDatabase())

[Table(name='department', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='employees', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
# to veiw glocal tem tables
spark.catalog.listTables('global_temp')

[Table(name='deptglobal', database='global_temp', description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='empglobal', database='global_temp', description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='department', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='employees', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [20]:
# to drop the temp view
spark.catalog.dropGlobalTempView('empGlobal')

True

# user Defined Functions UDFs

These are similar to functions in sql. We define some sogic in functions and store them in Database and use them in queries

Similar to that we can write our own custom logic in python function and register in with pyspark using udf() function



In [23]:
emp.show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

In [24]:
# now we want to have bonus of 5% for dept 10 , 10% for Dep 20 and 15% for dep 30
# if dept == 10 sal*(5/100) , if dept == 20 sal*(10/100) etc

In [38]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
def totalBonus(s,d):
    if s == 10:
        bpct = s*0.05
    elif s == 20:
        bpct = s* 0.1
    else:
        bpct = s* 0.15
    
    return bpct


bonus = udf(lambda s,d:totalBonus(s,d),FloatType())

In [65]:
#using this we are calculating bonus for each employee based on their department and salary
# for dept 10 it is 5%, for Dept 20 it is 10% and for Dept 30 it is 15%
emp_bonus = emp.withColumn('totalBonus',bonus(emp.SAL,emp.DEPTNO))
emp_bonus.show()

+-----+------+---------+----+-----------+----+----+------+----------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|totalBonus|
+-----+------+---------+----+-----------+----+----+------+----------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|     120.0|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|     240.0|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|     187.5|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|    446.25|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|     187.5|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|     427.5|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|     367.5|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|     450.0|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|     750.0|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|     225.0|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|     165.0|
| 7900| JAMES|    CL

In [43]:
#we can register the function to UDF at the time of creation in the follwing way

@udf(returnType=FloatType())
def totalBonus(s,d):
    if s == 10:
        bpct = s*0.05
    elif s == 20:
        bpct = s* 0.1
    else:
        bpct = s* 0.15
    
    return bpct

In [46]:
emp.select('*',totalBonus(emp.SAL,emp.DEPTNO).alias("Bonus")).show()

+-----+------+---------+----+-----------+----+----+------+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO| Bonus|
+-----+------+---------+----+-----------+----+----+------+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20| 120.0|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30| 240.0|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30| 187.5|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|446.25|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30| 187.5|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30| 427.5|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10| 367.5|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20| 450.0|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10| 750.0|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30| 225.0|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20| 165.0|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30| 142.5|
| 7902|  F

# RDD to dataFrame

In [47]:
data = [(1,'murali'),(2,'wafa')]
rdd = spark.sparkContext.parallelize(data)
print(rdd.collect())

[(1, 'murali'), (2, 'wafa')]


In [53]:
df = rdd.toDF(schema = ['id','name'])

In [54]:
df.show()

+---+------+
| id|  name|
+---+------+
|  1|murali|
|  2|  wafa|
+---+------+



In [56]:
df1 = spark.createDataFrame(rdd,schema = ['id','name'])
df1.show()

+---+------+
| id|  name|
+---+------+
|  1|murali|
|  2|  wafa|
+---+------+



# map()

its RDD tranformation used to apply function(lambda) on every element of RDD and returns new RDD

DataFrame don't support map() transformation to use with dataframe we need to generate RDD frist

In [60]:

data = [('reddy','pujari'),('geetha','tiruvaipati')]
rdd = spark.sparkContext.parallelize(data)


In [63]:
rdd1 = rdd.map(lambda x:x+(x[0]+ ' ' +x[1],))
print(rdd1.collect())

[('reddy', 'pujari', 'reddy pujari'), ('geetha', 'tiruvaipati', 'geetha tiruvaipati')]


In [64]:
emp.show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

# partitionBy()

In [66]:
emp_bonus.show()

+-----+------+---------+----+-----------+----+----+------+----------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|totalBonus|
+-----+------+---------+----+-----------+----+----+------+----------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|     120.0|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|     240.0|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|     187.5|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|    446.25|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|     187.5|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|     427.5|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|     367.5|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|     450.0|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|     750.0|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|     225.0|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|     165.0|
| 7900| JAMES|    CL

In [90]:
path = 'partitions/'

#help(emp.write.parquet)
emp.write.parquet(path=path,mode='overwrite',partitionBy='DEPTNO')

In [84]:
help(emp.write.csv)

Help on method csv in module pyspark.sql.readwriter:

csv(path: str, mode: Optional[str] = None, compression: Optional[str] = None, sep: Optional[str] = None, quote: Optional[str] = None, escape: Optional[str] = None, header: Union[bool, str, NoneType] = None, nullValue: Optional[str] = None, escapeQuotes: Union[bool, str, NoneType] = None, quoteAll: Union[bool, str, NoneType] = None, dateFormat: Optional[str] = None, timestampFormat: Optional[str] = None, ignoreLeadingWhiteSpace: Union[bool, str, NoneType] = None, ignoreTrailingWhiteSpace: Union[bool, str, NoneType] = None, charToEscapeQuoteEscaping: Optional[str] = None, encoding: Optional[str] = None, emptyValue: Optional[str] = None, lineSep: Optional[str] = None) -> None method of pyspark.sql.readwriter.DataFrameWriter instance
    Saves the content of the :class:`DataFrame` in CSV format at the specified path.
    
    .. versionadded:: 2.0.0
    
    Parameters
    ----------
    path : str
        the path in any Hadoop support

In [100]:
spark.read.parquet('partitions/DEPTNO=10').show()

+-----+------+---------+----+-----------+----+----+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|
+-----+------+---------+----+-----------+----+----+
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|
+-----+------+---------+----+-----------+----+----+



In [102]:
spark.read.parquet('partitions/').show()

+-----+------+---------+----+-----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+-----------+----+----+------+
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    1

# from_json()

In [111]:
data = (('murali',"{'hair' :'black', 'eye':'brown' }"), ('geetha',"{'hair' :'brown', 'eye':'blue' }"))

schema = ['id','props']

df = spark.createDataFrame(data,schema)
df.show(truncate=False)
df.printSchema()

+------+---------------------------------+
|id    |props                            |
+------+---------------------------------+
|murali|{'hair' :'black', 'eye':'brown' }|
|geetha|{'hair' :'brown', 'eye':'blue' } |
+------+---------------------------------+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)



In [113]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

structTypeSchema = StructType([\
    StructField('hair',StringType()),\
    StructField('eye',StringType()) \
    ] )

df1 = df.withColumn('propsSturct', from_json(df.props,structTypeSchema))
df1.show(truncate=False)

df1.printSchema()

+------+---------------------------------+--------------+
|id    |props                            |propsSturct   |
+------+---------------------------------+--------------+
|murali|{'hair' :'black', 'eye':'brown' }|{black, brown}|
|geetha|{'hair' :'brown', 'eye':'blue' } |{brown, blue} |
+------+---------------------------------+--------------+

root
 |-- id: string (nullable = true)
 |-- props: string (nullable = true)
 |-- propsSturct: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)



In [118]:
df1.withColumn('hair',df1.propsSturct.hair)\
.withColumn('eye',df1.propsSturct.eye).\
show()

+------+--------------------+--------------+-----+-----+
|    id|               props|   propsSturct| hair|  eye|
+------+--------------------+--------------+-----+-----+
|murali|{'hair' :'black',...|{black, brown}|black|brown|
|geetha|{'hair' :'brown',...| {brown, blue}|brown| blue|
+------+--------------------+--------------+-----+-----+

